In [42]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *

base = BaseOverlay("nightmode.bit")
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

In [43]:
hdmi_in.configure(PIXEL_RGBA)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGBA)

hdmi_in.start()
hdmi_out.start()

In [ ]:
hdmi_in.tie(hdmi_out)

In [44]:
from pynq import MMIO
#filter_reference = MMIO(0x00000000,0x10000)
filter_reference = MMIO(0x83C20000,0x10000)  #MMIO Addresses should always be double-checked when exporting a Vivado Design

In [ ]:
import cv2

for i in range(1000):
    in_frame = hdmi_in.readframe()
    out_frame = hdmi_out.newframe()
    
    filter_reference.write(0x10, in_frame.physical_address)   # Get Pointers to memory
    filter_reference.write(0x18, out_frame.physical_address)
    
    
    
    filter_reference.write(0x20, 1920)   # Make sure that the input HDMI signal is set to 1280x720
    filter_reference.write(0x28, 1080)
    #filter_reference.write(0x20, in_frame.shape[1]) 
    #filter_reference.write(0x28, in_frame.shape[0])
    filter_reference.write(0x00, 0x01)             # ap_start triggering
    while (filter_reference.read(0) & 0x4) == 0:   # ap_done checking
        pass
    #print(out_frame)
    cv2.rectangle(in_frame, (ElmoX-ElmoW,ElmoY), (ElmoX,ElmoY), (255,0,0), 2)
    hdmi_out.writeframe(out_frame)

# NEW CODE  

In [ ]:
import cv2
import random
from pynq import Xlnk
import numpy


xlnk = Xlnk()
vararray = xlnk.cma_array(shape = 6, dtype = np.int32)

imgCookie = cv2.imread('Cookie.png', -1)   #Loading Images
#imgCookie - cv2.cvtColor(imgCookie, cv2.COLOR_BGR2RGBA)
imgHeart = cv2.imread("heart.png", -1)
#imgHeart - cv2.cvtColor(imgHeart, cv2.COLOR_BGR2RGBA)
imgBroc = cv2.imread('Broccoli.png', -1)
#imgBroc - cv2.cvtColor(imgBroc, cv2.COLOR_BGR2RGBA)


frame_count = 0   #used to create retro looking speed

cookie_point_across_one = random.randint(50, 1200)   #spawn in objects
cookie_point_across_two = random.randint(50,1200)
cookie_point_across_three = random.randint(50,1200)
broc_point_across_one = random.randint(50,1200)
broc_point_across_two = random.randint(50,1200)

        
allgood = True
        
while(allgood):
    if abs(cookie_point_across_two-cookie_point_across_one) < 75:   #checking no collisions between cookies
        cookie_point_across_two = random.randint(50,1200)
    if abs(cookie_point_across_one-cookie_point_across_three) < 75:
        cookie_point_across_three = random.randint(50,1200)
    if abs(cookie_point_across_two-cookie_point_across_three) < 75:
        cookie_point_across_three = random.randint(50,1200)
    else:
        allgood = False
            
allgood = True

while(allgood):   #checking no collisions between cookies and broc
    if abs(broc_point_across_one-cookie_point_across_one) < 75 or abs(broc_point_across_one-cookie_point_across_two) < 75 or abs(broc_point_across_one-cookie_point_across_three) < 75:
        broc_point_across_one = random.randint(50,1200)
    if abs(broc_point_across_two-cookie_point_across_one) < 75 or abs(broc_point_across_two-cookie_point_across_two) < 75 or abs(broc_point_across_two-cookie_point_across_three) < 75:
        broc_point_across_two = random.randint(50,1200)
    if abs(broc_point_across_two-broc_point_across_one) < 75:
        broc_point_across_two = random.randint(50,1200)
    else:
        allgood = False
            
cookie_point_down_one = 0    #spawning cookies at the top
cookie_point_down_two = 0   
cookie_point_down_three = 0
broc_point_down_one = 0
broc_point_down_two = 0
#higher is faster
cookie_speed = 2  
lives_one = 3
lives_two = 3
score_one = 0
score_two = 0

player_one_x = 50
player_one_y = 600         #these will change with inputs from the cpp code
player_one_length = 400

player_two_x = 600
player_two_y = 600
player_two_length = 400

for _ in range(1000000):
    in_frame = hdmi_in.readframe()
    frame = hdmi_out.newframe()
    filter_reference.write(0x10, in_frame.physical_address)   # Get Pointers to memory
    filter_reference.write(0x18, frame.physical_address)
    filter_reference.write(0x20, vararray.physical_address)


    filter_reference.write(0x00, 0x01)             # ap_start triggering
    while (filter_reference.read(0) & 0x4) == 0:   # ap_done checking
        pass
    player_one_x = vararray[0]
    player_one_y = vararray[1] #these will change with inputs from the cpp code
    player_one_length = vararray[2]
    
    player_two_x = vararray[3]
    player_two_y = vararray[4]
    player_two_length = vararray[5]
    
    two_players = True
    
    if player_one_x == 0 or player_two_x == 0:
        two_players = False

    
    #setting up the two players
     
    player_one_x_end =  player_one_x-player_one_length       
    player_two_x_end =  player_two_x-player_two_length
    
    player_one_middle = player_one_x+(0.5*player_one_length)
    player_two_middle = player_two_x+(0.5*player_two_length)
    
    cv2.circle(frame, (vararray[3],vararray[4]), 10, (0,0,255), 1, 8, 0) #cookiemonster
    cv2.circle(frame, (vararray[0],vararray[1]), 10, (255,0,0), 1, 8, 0) #elmo
         
    cv2.line(frame,(player_one_x, player_one_y), (player_one_x_end, player_one_y), (0,0,255), 2)    #don't have these in final game
    cv2.line(frame,(player_two_x, player_two_y), (player_two_x_end, player_two_y), (255,0,0), 2)    #will have puppets, just for testing
    
    #Setting up variables for dimensions of objects
    
    cookie_h, cookie_w, cookie_c = imgCookie.shape
    broc_h, broc_w, broc_c = imgBroc.shape
    
    broc_bottom_one = broc_point_down_one + broc_h
    broc_bottom_two = broc_point_down_two + broc_h
    
    broc_left_one = broc_point_across_one + broc_w
    broc_left_two = broc_point_across_two + broc_w
    
    broc_middle_one = broc_bottom_one-(0.5*broc_h)
    broc_middle_two = broc_bottom_two-(0.5*broc_h)
    
    cookie_bottom_one = cookie_h + cookie_point_down_one
    cookie_bottom_two = cookie_h + cookie_point_down_two
    cookie_bottom_three = cookie_h + cookie_point_down_three
    
    cookie_left_one = cookie_point_across_one + cookie_w
    cookie_left_two = cookie_point_across_two + cookie_w
    cookie_left_three = cookie_point_across_three + cookie_w
    
    cookie_middle_y_one = cookie_bottom_one-(0.5*cookie_h)
    cookie_middle_y_two = cookie_bottom_two-(0.5*cookie_h)
    cookie_middle_y_three = cookie_bottom_three-(0.5*cookie_h)
    
    cookie_middle_x_one = cookie_point_across_one+(0.5*cookie_w)
    cookie_middle_x_two = cookie_point_across_two+(0.5*cookie_w)
    cookie_middle_x_three = cookie_point_across_three+(0.5*cookie_w)
    
    #Setting up Collisions with player_one
    
    broc_coll_one = False
    broc_coll_two = False

    if  (cookie_point_across_one) >= (player_one_x-30-player_one_length) and (cookie_left_one) <= (player_one_x+30) and abs(cookie_middle_y_one-player_one_y) < 37 :
        score_one = score_one + 10
        cookie_point_down_one = 0
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_one = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_one-cookie_point_across_two) > 75 and abs(cookie_point_across_one-cookie_point_across_three) > 75:
                    allgood = False
                else:
                    cookie_point_across_one = random.randint(50,1200)

    if (cookie_point_across_two) >= (player_one_x-30-player_one_length) and (cookie_left_two) <= (player_one_x+30) and abs(cookie_middle_y_two-player_one_y) < 37 :
        score_one = score_one + 10
        cookie_point_down_two = 0
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_two = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_two-cookie_point_across_one) > 75 and abs(cookie_point_across_two-cookie_point_across_three) > 75:
                    allgood = False
                else:
                    cookie_point_across_two = random.randint(50,1200)
                    
    if (cookie_point_across_three) >= (player_one_x-30-player_one_length) and (cookie_left_three) <= (player_one_x+30) and abs(cookie_middle_y_three-player_one_y) < 37 :
        score_one = score_one + 10
        cookie_point_down_three = 0
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_three = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_three-cookie_point_across_one) > 75 and abs(cookie_point_across_three-cookie_point_across_two) > 75:
                    allgood = False
                else:
                    cookie_point_across_three = random.randint(50,1200)
                    
    if (broc_point_across_one) >= (player_one_x-30-player_one_length) and (broc_left_one) <= (player_one_x+30) and abs(broc_middle_one-player_one_y) < 37 :
        lives_one = lives_one - 1
        broc_point_down_one = 0
        allgood = True
        broc_coll_one = True
        while(allgood):
            broc_point_across_one = random.randint(50,1200)
            if abs(broc_point_across_one-cookie_point_across_one) > 75 and abs(broc_point_across_one-cookie_point_across_two) > 75 and abs(broc_point_across_one-cookie_point_across_three) > 75:
                allgood = False
            else:
                broc_point_across_one = random.randint(50,1200)
                
    if (broc_point_across_two) >= (player_one_x-30-player_one_length) and (broc_left_two) <= (player_one_x+30) and abs(broc_middle_two-player_one_y) < 37 :
        lives_one = lives_one - 1
        broc_point_down_two = 0
        broc_coll_two = True
        allgood = True
        while(allgood):
            broc_point_across_two = random.randint(50,1200)
            if abs(broc_point_across_two-cookie_point_across_one) > 75 and abs(broc_point_across_two-cookie_point_across_two) > 75 and abs(broc_point_across_two-cookie_point_across_three) > 75:
                allgood = False
            else:
                broc_point_across_two = random.randint(50,1200)
                
    if broc_coll_one:
        if abs(broc_point_across_two-broc_point_across_one) < 75:
            broc_point_across_one = random.randint(50,1200)
            
    if broc_coll_two:
        if abs(broc_point_across_two-broc_point_across_one) < 75:
            broc_point_across_two = random.randint(50,1200)

    #Setting up Collisions with player_two
    
    broc_coll_one = False
    broc_coll_two = False

    if (cookie_point_across_one) > (player_two_x-30-player_two_length) and (cookie_left_two) < (player_two_x+30) and abs(cookie_middle_y_one-player_one_y) < 37 :
        score_two = score_two + 10
        cookie_point_down_one = 0
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_one = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_one-cookie_point_across_two) > 75 and abs(cookie_point_across_one-cookie_point_across_three) > 75:
                    allgood = False
                else:
                    cookie_point_across_one = random.randint(50,1200)

    if (cookie_point_across_two) > (player_two_x-30-player_two_length) and (cookie_left_two) < (player_two_x+30) and abs(cookie_middle_y_two-player_two_y) < 37 :
        score_two = score_two + 10
        cookie_point_down_two = 0
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_two = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_two-cookie_point_across_one) > 75 and abs(cookie_point_across_two-cookie_point_across_three) > 75:
                    allgood = False
                else:
                    cookie_point_across_two = random.randint(50,1200)
                    
    if (cookie_point_across_three) > (player_two_x-30-player_two_length) and (cookie_left_three) <= (player_two_x+30) and abs(cookie_middle_y_three-player_two_y) < 37:
        score_two = score_two + 10
        cookie_point_down_three = 0
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_threed = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_three-cookie_point_across_one) > 75 and abs(cookie_point_across_three-cookie_point_across_two) > 75:
                    allgood = False
                else:
                    cookie_point_across_three = random.randint(50,1200)
                    
    if (broc_point_across_one) > (player_two_x-30-player_two_length) and (broc_left_one) <= (player_two_x+30) and abs(broc_middle_one-player_two_y) < 37 :
        lives_two = lives_two - 1
        broc_point_down_one = 0
        allgood = True
        broc_coll_one = True
        while(allgood):
            broc_point_across_one = random.randint(50,1200)
            if abs(broc_point_across_one-cookie_point_across_one) > 75 and abs(broc_point_across_one-cookie_point_across_two) > 75 and abs(broc_point_across_one-cookie_point_across_three) > 75:
                allgood = False
            else:
                broc_point_across_one = random.randint(50,1200)
                
    if (broc_point_across_two) > (player_two_x-30-player_two_length) and (broc_left_two) <= (player_two_x+30) and abs(broc_middle_two-player_two_y) < 37 :
        lives_two = lives_two - 1
        broc_point_down_two = 0
        allgood = True
        broc_coll_two = True
        while(allgood):
            broc_point_across_two = random.randint(50,1200)
            if abs(broc_point_across_two-cookie_point_across_one) > 75 and abs(broc_point_across_two-cookie_point_across_two) > 75 and abs(broc_point_across_two-cookie_point_across_three) > 75:
                allgood = False
            else:
                broc_point_across_two = random.randint(50,1200)
                
    if broc_coll_one:
        if abs(broc_point_across_two-broc_point_across_one) < 75:
            broc_point_across_one = random.randint(50,1200)
            
    if broc_coll_two:
        if abs(broc_point_across_two-broc_point_across_one) < 75:
            broc_point_across_two = random.randint(50,1200)

            
    #Setting up speed
    
    
    cookie_point_down_one = cookie_point_down_one+cookie_speed
    cookie_point_down_two = cookie_point_down_two+cookie_speed
    cookie_point_down_three = cookie_point_down_three+cookie_speed
    broc_point_down_one = broc_point_down_one+cookie_speed
    broc_point_down_two = broc_point_down_two+cookie_speed

        
    #Setting up bottom collision
    
    broc_coll_one = False
    broc_coll_two = False

        
    if cookie_bottom_one >= 680:
        distance_one = ((player_one_y-cookie_middle_y_one)**2)+((player_one_middle-cookie_middle_x_one)**2)
        distance_two = ((player_two_y-cookie_middle_y_one)**2)+((player_two_middle-cookie_middle_x_one)**2)
        if two_players == True:
            if distance_one<distance_two:
                lives_one = lives_one-1
            if distance_one>distance_two:
                lives_two = lives_two -1
        if two_players == False:
            lives_one = lives_one-1
            lives_two = lives_two -1
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_one = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_one-cookie_point_across_two) > 75 and abs(cookie_point_across_one-cookie_point_across_three) > 75:
                    allgood = False
                else:
                    cookie_point_across_one = random.randint(50,1200)
            cookie_point_down_one = 0
    
    if cookie_bottom_two >= 690:
        distance_one = ((player_one_y-cookie_middle_y_two)**2)+((player_one_middle-cookie_middle_x_two)**2)
        distance_two = ((player_two_y-cookie_middle_y_two)**2)+((player_two_middle-cookie_middle_x_two)**2)
        if two_players == True:
            if distance_one<distance_two:
                lives_one = lives_one-1
            if distance_one>distance_two:
                lives_two = lives_two -1
        if two_players == False:
            lives_one = lives_one-1
            lives_two = lives_two -1
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_two = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_two-cookie_point_across_one) > 75 and abs(cookie_point_across_two-cookie_point_across_three) > 75:
                    allgood = False
                else:
                    cookie_point_across_three = random.randint(50,1200)
            cookie_point_down_two = 0
            
    if cookie_bottom_three >= 700:
        distance_one = ((player_one_y-cookie_middle_y_three)**2)+((player_one_middle-cookie_middle_x_three)**2)
        distance_two = ((player_two_y-cookie_middle_y_three)**2)+((player_two_middle-cookie_middle_x_three)**2)
        if two_players == True:
            if distance_one<distance_two:
                lives_one = lives_one-1
            if distance_one>distance_two:
                lives_two = lives_two -1
        if two_players == False:
            lives_one = lives_one-1
            lives_two = lives_two -1
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_three = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_three-cookie_point_across_one) > 75 and abs(cookie_point_across_three-cookie_point_across_two) > 75:
                    allgood = False
                else:
                    cookie_point_across_three = random.randint(50,1200)
            cookie_point_down_three = 0
            
    if broc_bottom_one >= 690:
        allgood = True
        broc_coll_one = True
        broc_point_across_one = random.randint(50,1200)
        while(allgood):
            if abs(broc_point_across_one-cookie_point_across_one) > 75 and abs(broc_point_across_one-cookie_point_across_two) > 75 and abs(broc_point_across_one-cookie_point_across_three) > 75:
                allgood = False
            else:
                broc_point_across_one = random.randint(50,1200)
        broc_point_down_one = 0
        
    if broc_bottom_two >= 690:
        allgood = True
        broc_coll_two = True
        broc_point_across_two = random.randint(50,1200)
        while(allgood):
            if abs(broc_point_across_two-cookie_point_across_one) > 75 and abs(broc_point_across_two-cookie_point_across_two) > 75 and abs(broc_point_across_two-cookie_point_across_three) > 75:
                allgood = False
            else:
                broc_point_across_two = random.randint(50,1200)
        broc_point_down_two = 0
        
    if broc_coll_one:
        if abs(broc_point_across_two-broc_point_across_one) < 75:
            broc_point_across_one = random.randint(50,1200)
            
    if broc_coll_two:
        if abs(broc_point_across_two-broc_point_across_one) < 75:
            broc_point_across_two = random.randint(50,1200)


        
    
    heart_h, heart_w, heart_c = imgHeart.shape
    
    #Printing the hearts
    
    while (lives_one <= 0):
        frame = hdmi_in.readframe()
        font = cv2.FONT_HERSHEY_TRIPLEX
        cv2.putText(frame, 'GAME OVER - ELMO WINS', (70, 400), font, 2, (255,255,255), 2, cv2.LINE_AA)
        hdmi_out.writeframe(frame)
        if base.buttons[0].read == 1:
            print("hi")
            score_one = 0
            score_two = 0
            cookie_point_down_one = 0
            cookie_point_down_two = 0
            cookie_point_down_three = 0
            broc_point_down_one = 0
            broc_point_down_two = 0
            lives_two = 3
            lives_one = 3


        
    while (lives_two <= 0):
        frame = hdmi_in.readframe()
        font = cv2.FONT_HERSHEY_TRIPLEX
        cv2.putText(frame, 'GAME OVER - COOKIE MONSTER WINS', (30, 400), font, 2, (255,255,255), 2, cv2.LINE_AA)
        hdmi_out.writeframe(frame)
        if base.buttons[0].read == 1:
            score_one = 0
            score_two = 0
            cookie_point_down_one = 0
            cookie_point_down_two = 0
            cookie_point_down_three = 0
            broc_point_down_one = 0
            broc_point_down_two = 0
            lives_one = 3
            lives_two = 3  
    
    frame_h, frame_w, frame_c = frame.shape
    
    cv2.rectangle(frame, (1280,0), (1130,50), (0,0,0), -1)
    cv2.rectangle(frame, (1280,70), (1130,120), (0,0,0), -1)
    
    
    y_off=0
    if lives_one == 3:
        x_off=1230
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart   
        x_off=1180
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart       
        x_off=1130
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart
                
                
    if lives_one == 2:
        x_off=1230
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart   
        x_off=1180
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart 
        
            
     
    if lives_one == 1:
        x_off=1230
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart 
        
    y_off=70
    if lives_two == 3:
        x_off=1230
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart   
        x_off=1180
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart       
        x_off=1130
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart
                
                
    if lives_two == 2:
        x_off=1230
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart   
        x_off=1180
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart 
        
            
     
    if lives_two == 1:
        x_off=1230
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart 
     
        
    #Printing the cookies and broc
    
    if lives_one > 0 and lives_two > 0:
        x1, x2 = cookie_point_across_one, cookie_point_across_one + imgCookie.shape[1]
        y1, y2 = cookie_point_down_one, cookie_point_down_one + imgCookie.shape[0]
        alpha_cookie = imgCookie[:,:, 3] / 255.0
        alpha_frame = 1.0 - alpha_cookie
        for c in range(0, 3):
            frame[y1:y2, x1:x2, c] = (alpha_cookie*imgCookie[:,:,c] + alpha_frame*frame[y1:y2, x1:x2, c])
        x1, x2 = cookie_point_across_two, cookie_point_across_two + imgCookie.shape[1]
        y1, y2 = cookie_point_down_two, cookie_point_down_two + imgCookie.shape[0]
        alpha_cookie = imgCookie[:,:, 3] / 255.0
        alpha_frame = 1.0 - alpha_cookie
        for c in range(0, 3):
            frame[y1:y2, x1:x2, c] = (alpha_cookie*imgCookie[:,:,c] + alpha_frame*frame[y1:y2, x1:x2, c]) 
        x1, x2 = cookie_point_across_three, cookie_point_across_three + imgCookie.shape[1]
        y1, y2 = cookie_point_down_three, cookie_point_down_three + imgCookie.shape[0]
        alpha_cookie = imgCookie[:,:, 3] / 255.0
        alpha_frame = 1.0 - alpha_cookie
        for c in range(0, 3):
            frame[y1:y2, x1:x2, c] = (alpha_cookie*imgCookie[:,:,c] + alpha_frame*frame[y1:y2, x1:x2, c]) 
                
        x1, x2 = broc_point_across_one, broc_point_across_one + imgBroc.shape[1]
        y1, y2 = broc_point_down_one, broc_point_down_one + imgBroc.shape[0]
        alpha_broc = imgBroc[:,:, 3] / 255.0
        alpha_frame = 1.0 - alpha_broc
        for c in range(0, 3):
            frame[y1:y2, x1:x2, c] = (alpha_broc*imgBroc[:,:,c] + alpha_frame*frame[y1:y2, x1:x2, c]) 
        x1, x2 = broc_point_across_two, broc_point_across_two + imgBroc.shape[1]
        y1, y2 = broc_point_down_two, broc_point_down_two + imgBroc.shape[0]
        alpha_broc = imgBroc[:,:, 3] / 255.0
        alpha_frame = 1.0 - alpha_broc
        for c in range(0, 3):
            frame[y1:y2, x1:x2, c] = (alpha_broc*imgBroc[:,:,c] + alpha_frame*frame[y1:y2, x1:x2, c]) 

          
    cv2.putText(frame, str(score_one), (70,50), cv2.FONT_HERSHEY_TRIPLEX, 2, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(frame, str(score_two), (70,100), cv2.FONT_HERSHEY_TRIPLEX, 2, (0,0,0), 1, cv2.LINE_AA)   
    
    cv2.putText(frame, "COOKIE MONSTER", (400,50), cv2.FONT_HERSHEY_TRIPLEX, 1, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(frame, "ELMO", (400,100), cv2.FONT_HERSHEY_TRIPLEX, 1, (0,0,0), 1, cv2.LINE_AA)
        

    
    frame_count = frame_count + 1
    hdmi_out.writeframe(frame)
    





In [41]:
hdmi_in.close()    # Don't forget to run this to free memory
hdmi_out.close()   # NEVERFORGET NEVERFORGET NEVERFORGET :p 